# DQN (Frozen Lake)
---
### 환경
---
* Q, V 는 샘플링을 하는 순간 추정값이 됨
    * 샘플을 무한대로 모으면 true 로 수렴함
    * 신경망을 쓰지 않는 MC, TD 는 그냥 이 샘플을 true 라고 가정하고 대체해버림
    * 그래서 true 를 따로 고려하지 않음
    * 그러나 신경망을 쓰는 순간, 근사(추정) 값과 true 를 비교하기 때문에 true 에 대해서 정확히 알아야 함
    * 여기서 true 는 대체된 pseudo true 임
* 그런데 신경망 근사기로 학습을 하면 추정을 한 단계 더 추가하게 됨
* 더더 부정확해지고 불안정해짐
* q(s, a) 를 q(s, a, w) 로 바꿔 주어진 데이터보다 훨씬 더 적은 파라미터에 의해 압축시킴
    * 이게 ML 이 하는 일
* q 표현
    * 입력으로는 s, a 가 들어가고 그 안에서 w 를 거쳐서 q 값이 나오는게 설명
    * 실제로는 s 만 들어가고 각 a 에 대한 q 값 출력으로 구조 변환 -> 신경망 라이브러리 구현에 용이
    * 이로 미루어보면 기본적으로, q 는 discrete action space 에 유효함
    * 대신 입력 부분인 state 는 continous 하게 바꿀 수 있음
* 목적 함수
    * 근사 방법을 RL 과 연결하려면 예측값과 정답값의 차이를 정의해야 함 -> true Q (sampled pseudo Q), pred Q
    * true 라고 했지만 사실은 부족한 true 이면서 또 그걸 근사까지 하니까 상당히 불안정해짐
    * 업데이트 식은 그대로 두고 Q 의 error 를 최적화 대상으로 학습만 시킴
    * error = Q_target - Q, Q_target = R + gamma*Q'
    * 두 개의 Q 모두 신경망에서 나온 값 들임.
    * 기존의 RL 알고리즘의 error 반영 방식을 그대로 신경망을 활용하여 적용
    * 근데 TD 방식이므로 한 step 에 대한 error 에 해당함
    * 결국 Q 라는 자료구조, 함수를 업데이트 시키는건데 그게 신경망으로 표현하게 변경한것 뿐임
    * Q 는 w 라는 파라미터들로 표현된 신경망.
    * 기존의 Q 는 모든 s, a 를 요소로 갖고 있지만 w 로 압축 표현하고 있음.
    * 그래서 명시적으로 table 의 개별 cell 의 Q 값을 업데이트 하는것이 아니라, w 뭉치를 업데이트 한다
    * table q 는 무식하게 값을 대입하는데, 신경망 q 는 optimizer 가 고차원 함수의 경사를 타고 이동하며 변화시킴
    * 이렇게 모델 (신경망, 기타 ML 모델) 들이 강화학습 문제에 plug-in 됨
* target Q
    * 분명히 Q 러닝 자체가 off policy 인데 굳이 또 target Q 를 분리하는 이유는?
    * q 러닝은 q(s, a) 테이블의 cell 값 하나를 업데이트 하면 다른 cell 은 변하지 않음. 고정되어 있음
    * 그러나 신경망을 쓰면 cell 이 박혀있지 않고 w 로 표현되기 때문에 한 번 업데이트 하면 전체적으로 조금씩 움직이게 됨
    * 그래서 매번 움직여 고정되지 않으니 불안정하게 되어버림
    * 업데이트 식 자체에 2 개의 prediction 값의 차이가 포함되어 있어서 하나의 w 대해 업데이트의 방향이 난잡해지게 됨!
    * 그래서 table q 처럼 고정을 시키려면 아예 네트워크를 새로 하나 만들어서 일정기간 동안은 변하지 않는 참조값을 가질 필요가 있음
* off policy 강화
    * 리플레이 버퍼를 사용하면 원래 off policy 인 DQN 의 off 성향을 더 강화할 수 있음
    * 리플레이 버퍼에 과거 상호작용 데이터를 집어넣고 랜덤으로 불러왔을 때, 그 당시 정책과 현재 정책이 다르기 때문에 off policy 조건이 만족함
    * DQN 은 이미 off policy 인 q 러닝을 신경망 버전으로 옮긴 것
* off policy vs off line
    * off policy 는 행동 생성 정책과 데이터 수집 정책이 다른 상태
    * off line 은 아예 batch 데이터가 수집되고 더 이상 상호작용이 없는 상황
    * off line 은 데이터 수집이 차단됨
* 신경망을 RL 에 적용시키는 작업은 절대 단순한 작업이 아님.
    * 온갖 트릭을 덕지덕지 붙여야 그나마 잘 작동할 수 있는 복잡한 기술임

In [17]:
from keras.layers import Dense
from keras.models import Sequential
import numpy as np
import gym
from gym.envs.registration import register
from keras import optimizers
from keras import initializers
import random
from collections import deque

In [3]:
'''
환경셋팅 한 후에 환경을 추가등록한다.
'''

register(
    id='FrozenLake-v1',
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={'map_name':'4x4','is_slippery':False})

In [4]:
'''
환경 생성
'''
env = gym.make('FrozenLake-v1')

In [48]:
class DQN_Agent:
    def __init__(self, n_state, n_action):
        self.n_state = n_state
        self.n_action = n_action
        
        self.gamma = .95
        self.lr = .01
        self.epsilon = 1
        self.batch_size = 32
        self.train_start = 1000
        
        self.memory = deque(maxlen=2000)
        
        self.model = self.build_model()
        self.target_model = self.build_model()
        
        self.update_target_model()
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(8, input_dim=self.n_state))
        model.add(Dense(16))
        model.add(Dense(6))
        model.add(Dense(self.n_action, activation='sigmoid'))
        
        #model.summary()
        model.compile(loss='mse', optimizer=optimizers.Adam(self.lr))
    
        return model
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def append_sample(self, state, action, reward, state_next, done):
        self.memory.append((state, action, reward, state_next, done))
    
    def train_model(self):
        #if(self.epsilon > self.epsi)
        mini_batch = random.sample(self.memory, self.batch_size)
        states = np.zeros((self.batch_size, self.n_state))
        states_next = np.zeros((self.batch_size, self.n_state))
        actions, rewards, dones = [], [], []
        
        for i in range(self.batch_size):
            states[i] = mini_batch[i][0]
            actions.append(mini_batch[i][1])
            rewards.append(mini_batch[i][2])
            states_next[i] = mini_batch[i][3]
            dones.append(mini_batch[i][4])
            
        target = self.model.predict(states)
        target_val = self.target_model.predict(states_next)
        
        for i in range(self.batch_size):
            if dones[i]:
                target[i][actions[i]] = rewards[i]
            else:
                target[i][actions[i]] = rewards[i] + self.gamma * (np.amax(target_val[i]))
                
        self.model.fit(states, target, batch_size=self.batch_size, epochs=1, verbose=0)

In [55]:
episode = 0
max_episode = 300

state = env.reset()
action = env.action_space.sample()
step = 0

agent = DQN_Agent(env.observation_space.n, env.action_space.n)

for ep in range(max_episode):
    
    state = env.reset()
    state = np.eye(16)[state]
    state = np.reshape(state, [1, 16])
    done = False
    agent.epsilon -= .0033
    
    while not done:
        
        # action 선택
        if(random.random() > agent.epsilon):
            action = agent.model.predict(state)
            action = np.argmax(action)
            
        else:
            action = env.action_space.sample()
        
        # step 진행
        state_next, reward, done, _ = env.step(action)
        state_next = np.eye(16)[state_next]
        state_next = np.reshape(state_next, [1, 16])
        # 실패 경우 보상 처리
        if(done and reward < 1):
            reward = -1
        
        # sample 저장
        agent.append_sample(state, action, reward, state_next, done)
        
        # 학습
        if(agent.train_start < len(agent.memory)):
            agent.train_model()
    
env.close()

In [56]:
s = env.reset()
while(True):
    s = np.eye(16)[s]
    s = np.reshape(s, [1, 16])
    a = agent.model.predict(s)
    a = np.argmax(a)
    print(s)
    s, r, d, _ = env.step(a)
    if(d):
        env.render()
        break

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0

KeyboardInterrupt: 